In [1]:
import os
import sys 
import configparser
import pandas as pd
import numpy as np



config = configparser.ConfigParser()
config.read('../config.ini')
data_folder = config['paths']['path_data']

KeyError: 'paths'

### SQL queries 

In [2]:
import pyodbc





query_symbols_correspondence = """
        SELECT [sym_sedol_hist].[fsym_id]
              ,[bbg_ticker]
              ,[sedol]
              ,[fsym_security_id]
              ,[ce_sec_entity].factset_entity_id
              ,factset_ultimate_parent_entity_id
        FROM [fds].[sym_v1].[sym_sedol_hist]

        join [fds].[sym_v1].[sym_coverage]
        on [sym_coverage].fsym_id = [sym_sedol_hist].[fsym_id]

 

        join [fds].[sym_v1].[sym_bbg]
        on sym_bbg.fsym_id = sym_sedol_hist.fsym_id

 

        join [fds].[evt_v1].[ce_sec_entity]
        on [fsym_security_id] = [ce_sec_entity].fsym_id

 

        join [fds].[ent_v1].[ent_entity_structure]
        on [ce_sec_entity].factset_entity_id = [ent_entity_structure].factset_entity_id
    """

query_beta = """
SELECT  px.fsym_id
      , px.p_date
      , px.exch_rate_sec_eur
      , px.p_price_adj_split_spinoff_div
      , px.mcap_local
      ,sec.factset_industry
      ,sec.factset_sector
	  ,[04_Factor_Raw_Histo].[factor_value] as beta

  FROM [ScreenFactory].[dbo].[03_DailyPx_Temp] px

  left join [ScreenFactory].[dbo].[Securities_Fds] sec
  on px.fsym_id = sec.fsym_id

  left join [ScreenFactory].[dbo].[04_Factor_Raw_Histo] 
  on px.fsym_id = [04_Factor_Raw_Histo].fsym_id AND px.p_date = [04_Factor_Raw_Histo].p_date

  WHERE factor_raw_id = 255 
  order by px.p_date desc

"""

### Price Data

In [9]:
os.listdir(os.path.join(data_folder))

['news_200304_202212.pkl',
 'news_headlines_final_v2.1(raw).parquet',
 'news_headlines_v1.1.parquet',
 'news_headlines_final_v2_merged.parquet',
 'news_headlines_final_v1.2.parquet',
 'news_headlines_v1.2.parquet',
 'df_price.parquet',
 'news_headlines_v1.2_notmerged.parquet',
 'trscript_data_t_199905_202205.pkl',
 'news_headlines_final_v2.parquet',
 'df_price_v1.1.parquet']

In [39]:
import pyodbc
import pandas as pd

query_symbols_correspondence = """
        SELECT [sym_sedol_hist].[fsym_id]
              ,[bbg_ticker]
              ,[sedol]
              ,[fsym_security_id]
              ,[ce_sec_entity].factset_entity_id
              ,factset_ultimate_parent_entity_id
        FROM [fds].[sym_v1].[sym_sedol_hist]

        join [fds].[sym_v1].[sym_coverage]
        on [sym_coverage].fsym_id = [sym_sedol_hist].[fsym_id]

 

        join [fds].[sym_v1].[sym_bbg]
        on sym_bbg.fsym_id = sym_sedol_hist.fsym_id

 

        join [fds].[evt_v1].[ce_sec_entity]
        on [fsym_security_id] = [ce_sec_entity].fsym_id

 

        join [fds].[ent_v1].[ent_entity_structure]
        on [ce_sec_entity].factset_entity_id = [ent_entity_structure].factset_entity_id
    """

# Your connection string
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=RAMASRVSQL03.ramai.local;"
    "UID=linuxml;"
    "PWD=lymph-nBFKGPj5"
)
conn_str

# Initiate the connection
conn = pyodbc.connect(conn_str)

# Execute the query and read into a DataFrame
df_symbols = pd.read_sql(query_symbols_correspondence, conn)

# Close the connection
conn.close()

# Show the DataFrame
df_symbols.head(2)

/tmp/ipykernel_2584294/3186477113.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_symbols = pd.read_sql(query_symbols_correspondence, conn)


,fsym_id,bbg_ticker,sedol,fsym_security_id,factset_entity_id,factset_ultimate_parent_entity_id
0,B01CM9-R,None,B0187M3,TNZRTW-S,00193B-E,0P46GJ-E
1,B01Q8M-R,None,5943757,GV16Y2-S,05HPSL-E,05HPSL-E


In [40]:
list(filter(lambda st : 'price' in st, os.listdir(os.path.join(data_folder))))

['df_price.parquet', 'df_price_v1.1.parquet']

In [2]:
import pandas as pd
import numpy as np
import os 
import datetime, re


df_price = pd.read_parquet(os.path.join(data_folder, 'df_price_v1.1.parquet'))
df_price = df_price.rename(columns=  {'mcap_local': 'DES_MCAP_CCY', 'factset_industry': 'DES_FACTSET_INDUSTRY', 'factset_sector': 'DES_FACTSET_SECTOR'})


In [3]:
for col in df_price.columns:
    print(col)

exch_rate_sec_eur
p_price_adj_split_spinoff_div
DES_MCAP_CCY
DES_FACTSET_INDUSTRY
DES_FACTSET_SECTOR
DES_BETA_ADJ_104W
RET_5D_bin
RET_10D_bin


In [23]:

news_pkl = pd.read_pickle(os.path.join(data_folder, 'news_200304_202212.pkl'))
news_pkl.head(2)
for col in news_pkl.columns:
    print(col)

all_factset_entity_id
all_ticker_region
factset_entity_id
fds_country
fds_language
fds_subject
headline
headline_no_ent
last_update_time_stamp
product
publish_time
reference_story_headline
reference_story_id
reference_story_time
report_id
sa_category
schema_version
senti_headline
senti_headline_no_ent
story_body
story_date_time
ticker_region


In [25]:
cols_date = [col for col in news_pkl.columns if "date" in col.lower() or "time" in col.lower()]
cols_date
news_pkl[cols_date].iloc[:1000:100]

,last_update_time_stamp,publish_time,reference_story_time,story_date_time
0,None,2012-06-06T05:40:39Z,[],2003-10-01T00:58:49Z
100,None,2012-06-06T05:45:26Z,[],2003-10-01T12:09:55Z
200,None,2012-06-06T05:51:49Z,[],2003-10-01T18:37:50Z
300,None,2012-06-06T07:28:01Z,[],2003-10-10T11:07:16Z
400,None,2012-06-06T07:34:46Z,[],2003-10-10T17:45:00Z
500,None,2012-06-06T07:45:18Z,[],2003-10-13T11:11:30Z
600,None,2012-06-06T07:50:42Z,[],2003-10-13T16:47:51Z
700,None,2012-06-06T07:59:30Z,[],2003-10-14T10:19:50Z
800,None,2012-06-06T08:02:13Z,[],2003-10-14T12:40:43Z
900,None,2012-06-06T08:09:16Z,[],2003-10-14T20:08:07Z


In [26]:
#create SYMBOL index
print(f"creating SYMBOL index")
news_pkl = news_pkl.rename(columns = {"factset_entity_id": "SYMBOL", "story_date_time": "DATE"})
news_pkl = news_pkl.explode("SYMBOL").dropna(subset = ['SYMBOL']) # converts list of symbols into separate rows
news_pkl = news_pkl[['DATE', 'SYMBOL', 'headline']]
news_pkl.head(10)


creating SYMBOL index


,DATE,SYMBOL,headline
5,2003-10-01T02:30:05Z,000Q29-E,MMM reaffirmed outlook for 12-14% EPS growth i...
9,2003-10-01T09:13:42Z,00118C-E,ADTN initiated neutral at Merrill Lynch
14,2003-10-01T09:20:09Z,000MZR-E,IDTI initiated underweight at Lehman
15,2003-10-01T09:23:59Z,000FGN-E,CY upgraded to equal weight from underweight a...
18,2003-10-01T09:29:49Z,002Q0G-E,XOM could face new payments from Alaska Valdez...
21,2003-10-01T09:35:10Z,002RBQ-E,ADS upgraded to overweight from neutral at JP ...
22,2003-10-01T09:36:14Z,00C4H1-E,AOL expected to introduce a Spanish-language s...
23,2003-10-01T09:38:56Z,003FY6-E,WBS upgraded to sector perform from underperfo...
24,2003-10-01T09:39:56Z,0027L0-E,C CEO Charles Prince takes over helm today; pl...
26,2003-10-01T09:48:10Z,0091M7-E,BK upgraded to buy from neutral at UBS


In [27]:
# process DATE : convert 2012-06-06T05:40:39Z to 2012-06-06
print(f"processing DATE")
datecol = pd.to_datetime(news_pkl['DATE']).dt.tz_localize(None).dt.strftime('%Y-%m-%d')
news_pkl['DATE'] = pd.to_datetime(datecol)

#set the index to SYMBOL, DATE
print(f"setting index to SYMBOL, DATE")
news_pkl = news_pkl.set_index(['SYMBOL', 'DATE']).sort_index()

print(f"shifting dates by 1 day")
news_pkl['DATE2'] =( news_pkl.reset_index(level = 1)['DATE'] + datetime.timedelta(days=1)).values
news_pkl = news_pkl.reset_index(level = 1, drop = True).rename(columns = {"DATE2": "DATE"}).reset_index(drop=False).set_index(['SYMBOL', 'DATE']).sort_index()

print('removing "StreetAccount" reports ')
news_pkl.filter(regex = '.*Street\s*Account.*', axis = 0)
def filter_function(x:str):
    pattern = r'.*Street\s*Account.*'
    if not x.__class__ == str:
        return False
    return re.search(pattern, x) is not None
news_pkl['hasSA'] = news_pkl['headline'].apply(filter_function)
print("num 'StreetAccount' rows : ", news_pkl.hasSA.sum() / news_pkl.shape[0], "out of ", news_pkl.shape[0])
news_pkl = news_pkl.loc[~news_pkl.hasSA, ['headline']]

news_pkl['headline'] = news_pkl['headline'].str.replace(r'\bEPS\b', 'Earnings Per Share', regex=True)
news_pkl['headline'] = news_pkl['headline'].str.replace(r'[-]{1,2} [A-Z]\w*', '', regex=True)


def filter_function2(x:str):
    pattern = r'trading (higher|lower|up|down)\b'
    if not x.__class__ == str:
        return False
    return re.search(pattern, x.lower()) is not None
hasTrading = news_pkl['headline'].apply(filter_function2)

print("num 'Reports vs' rows : ", hasTrading.sum() / news_pkl.shape[0], "out of ", news_pkl.shape[0])
news_pkl = news_pkl.loc[~hasTrading, ["headline"]]



mask = news_pkl['headline'].str.contains(r"Barron's Summary|Street Takeaways", case=False, regex=True)
print(f"num rows with 'Barron's Summary|Street Takeaways' : ", mask.sum() / news_pkl.shape[0], "out of ", news_pkl.shape[0])
news_pkl = news_pkl[~mask]


news_pkl = news_pkl[news_pkl['headline'].apply(lambda x: len(x.split()) > 5)]


processing DATE
setting index to SYMBOL, DATE
shifting dates by 1 day
removing "StreetAccount" reports 
num 'StreetAccount' rows :  0.0564676870348456 out of  2679621


headline
SYMBOL   DATE                                                         
000BDX-E 2004-07-30  SRCE reports Q2 $0.42 vs. First Call $0.31 -- ...
         2004-10-29  SRCE reports Q3 $0.16 vs. First Call $0.33 -- ...

In [37]:
print(f"number of symbols : {len(news_pkl.index.get_level_values(0).unique())}")
print(f"number of dates : {len(news_pkl.index.get_level_values(1).unique())}")
print(f"number of headlines : {news_pkl.shape[0]} ie {news_pkl.shape[0] / 10**6 :.2f}M")

number of symbols : 26220
number of dates : 7152
number of headlines : 2381758 ie 2.38M


In [38]:

print(f"merging with df_symbols")
# merge -E and -R, and keep -R, but one -R can have multiple -Es
news_pkl= news_pkl.reset_index(drop=False).merge(df_symbols[['fsym_id', 'factset_entity_id']], left_on = 'SYMBOL', right_on = 'factset_entity_id', how = 'left')
del news_pkl['SYMBOL']
news_pkl.rename(columns = {"fsym_id": "SYMBOL"}, inplace = True)
if not "SYMBOL" in news_pkl.index.names and not "DATE" in news_pkl.index.names:
    news_pkl = news_pkl.set_index(['SYMBOL', 'DATE']).sort_index()

print(f"merging with df_price")
cols_final = ['SYMBOL', 'DATE', 'RET_5D_bin',  'RET_10D_bin','DES_MCAP_CCY','DES_FACTSET_INDUSTRY',	'DES_FACTSET_SECTOR'	,'DES_BETA_ADJ_104W']
out = news_pkl[['headline']].merge(
    df_price.reset_index(drop=False)[cols_final], 
    on = ['SYMBOL', 'DATE'],
    how = 'left').dropna(subset = ['RET_10D_bin', 'DES_BETA_ADJ_104W'])
diff = set(news_pkl.index.get_level_values(0)) - set(df_price.index.get_level_values(0))
print(f"number of NA symbols : {len(diff)} out of {len(set(news_pkl.index.get_level_values(0)))}")



merging with df_symbols


headline  \
SYMBOL   DATE                                                            
B006D6-R 2003-05-02  EEFT upgraded to outperform from mkt perform a...   
         2003-07-31  EEFT downgraded to market perform from outperf...   

                    factset_entity_id  
SYMBOL   DATE                          
B006D6-R 2003-05-02          001FXQ-E  
         2003-07-31          001FXQ-E

In [52]:
out.shape
print(f"number of dates {out.DATE.nunique()} and SYMBOLs {out.SYMBOL.nunique()}")
print(f"min date {out.DATE.min()} and max date {out.DATE.max()}")
print(f"shape {out.shape}")

number of dates 4333 and SYMBOLs 11100
min date 2003-04-11 00:00:00 and max date 2019-12-31 00:00:00
shape (1664327, 9)


In [53]:
out.to_parquet(os.path.join(data_folder, 'news_headlines_v3.parquet'))

In [54]:
def gen():
    for i in range(0, out.shape[0]):
        yield out.iloc[i]['RET_10D_bin'],out.iloc[i]['headline']
gene = gen()


In [65]:

out = out.reset_index(drop=False).drop_duplicates(subset = ['SYMBOL','DATE','headline', 'RET_5D_bin', 'RET_10D_bin'], keep = 'first')

In [69]:
del out['index']


In [70]:
if "SYMBOL" in out.columns and "DATE" in out.columns:
    out = out.set_index(['SYMBOL', 'DATE']).sort_index()
out.iloc[2000:2050:2]

headline  \
SYMBOL   DATE                                                            
B19ST9-R 2006-03-20  Barron's says all things considered LLY looks ...   
         2006-03-29  LLY downgraded to neutral from buy at Merrill ...   
         2006-04-13  LLY announces preliminary results for Evista (...   
         2006-04-18                UBS comments on Evista data for LLY   
         2006-04-21             Consensus drug sales estimates for LLY   
         2006-04-24   Bias suit filed against LLY reports the NY Times   
         2006-05-05           LLY responds to loss in ARIA patent case   
         2006-05-24  New study suggests Symbyax significantly impro...   
         2006-06-01  LLY launches Phase III trial of enzastaurin fo...   
         2006-06-07  LLY's Prozac receives positive opinion from Eu...   
         2006-06-14  LLY to seek approval for once a day Cialis  Times   
         2006-06-15  LLY's Prozac shows no benefit in treatment of ...   
         2006-06-23            BofA affirms buy after meeting with LLY   
         2006-07-06          LLY added to Americas Buy List at Goldman   
         2006-07-14  LLY's Evista (raloxifene) elevates blood clot,...   
         2006-08-01  FDA sends LLY a warning letter regarding Alimt...   
         2006-09-15  Eli Lilly and Amylin Pharma (AMLN) announce re...   
         2006-09-22  Eli Lilly/Amylin's (AMLN) Byetta receives posi...   
         2006-10-18    UBS comments on Eli Lilly's acquisition of ICOS   
         2006-10-20  Lilly says it will appeal FDA's decision on Ar...   
         2006-10-20  Three NIH doctors say Eli Lilly manipulated tr...   
         2006-10-24  Eli Lilly announces new study data showing pra...   
         2006-11-22  Eli Lilly, Amylin (AMLN) say EC has granted ap...   
         2006-12-06  Eli Lilly sues Indian-based Sun Pharmaceutical...   
         2006-12-08                Eli Lilly maintained neutral at UBS   

                     RET_5D_bin  RET_10D_bin  DES_MCAP_CCY  \
SYMBOL   DATE                                                
B19ST9-R 2006-03-20         1.0          0.0   64883623.86   
         2006-03-29         0.0          0.0   63640642.56   
         2006-04-13         0.0          0.0   60194194.41   
         2006-04-18         0.0          0.0   61584073.50   
         2006-04-21         0.0          0.0   59810000.19   
         2006-04-24         0.0          0.0   59391906.48   
         2006-05-05         0.0          1.0   59829135.14   
         2006-05-24         1.0          1.0   57342846.94   
         2006-06-01         1.0          1.0   58755510.69   
         2006-06-07         1.0          1.0   58439074.01   
         2006-06-14         0.0          0.0   60089065.27   
         2006-06-15         0.0          1.0   61083580.55   
         2006-06-23         0.0          0.0   61366113.30   
         2006-07-06         0.0          1.0   64044523.77   
         2006-07-14         0.0          1.0   61659947.36   
         2006-08-01         0.0          0.0   63615073.99   
         2006-09-15         1.0          1.0   61945865.20   
         2006-09-22         1.0          1.0   63223216.07   
         2006-10-18         0.0          0.0   65845741.75   
         2006-10-20         0.0          0.0   64704038.76   
         2006-10-20         0.0          0.0   64704038.76   
         2006-10-24         0.0          0.0   65314454.22   
         2006-11-22         0.0          0.0   61139699.64   
         2006-12-06         0.0          0.0   62078917.68   
         2006-12-08         0.0          0.0   61105752.00   

                       DES_FACTSET_INDUSTRY DES_FACTSET_SECTOR  \
SYMBOL   DATE                                                    
B19ST9-R 2006-03-20  Pharmaceuticals: Major  Health Technology   
         2006-03-29  Pharmaceuticals: Major  Health Technology   
         2006-04-13  Pharmaceuticals: Major  Health Technology   
         2006-04-18  Pharmaceuticals: Major  Health Techno

In [74]:
print( f"number of dates {out.index.get_level_values(1).nunique()} and SYMBOLs {out.index.get_level_values(0).nunique()}")
print(f"min date {out.index.get_level_values(1).min()} and max date {out.index.get_level_values(1).max()}")
print(f"shape {out.shape}")

number of dates 4333 and SYMBOLs 11100
min date 2003-04-11 00:00:00 and max date 2019-12-31 00:00:00
shape (1002870, 7)


In [71]:
next(gene)
from itertools import islice 

with open("news_headlines_v3.1.txt", "w") as f:
    for i in islice(range(0, out.shape[0]), 2000):
        f.write(str(out.iloc[i]['RET_10D_bin']) + "\t" + out.iloc[i]['headline'] + "\n")

In [75]:
out.to_parquet(os.path.join(data_folder, 'news_headlines_v3.parquet'))

### Changing target to binary positive negative

In [7]:
df_price = pd.read_parquet(os.path.join(data_folder, 'df_price.parquet'))
df_price.columns

Index(['exch_rate_sec_eur', 'p_price_adj_split_spinoff_div', 'DES_MCAP_CCY',
       'DES_FACTSET_INDUSTRY', 'DES_FACTSET_SECTOR', 'DES_BETA_ADJ_104W',
       'RET_5D', 'RET_5D_bin', 'RET_10D', 'RET_10D_bin'],
      dtype='object')

In [8]:
df_price['RET_5D_pos'] = (df_price['RET_5D'] > 0).astype(int)
df_price['RET_10D_pos'] = (df_price['RET_10D'] > 0).astype(int)
df_price[['RET_5D_pos', 'RET_10D_pos', 'RET_5D', 'RET_10D']].head(10)

RET_5D_pos  RET_10D_pos    RET_5D   RET_10D
SYMBOL   DATE                                                   
B00165-R 1998-01-01           1            0  0.011591 -0.056464
         1998-01-02           1            0  0.009476 -0.001882
         1998-01-05           1            1  0.000650  0.002199
         1998-01-06           1            1  0.007658  0.025806
         1998-01-07           0            0 -0.024011 -0.014027
         1998-01-08           0            0 -0.067275 -0.022539
         1998-01-09           0            0 -0.011251 -0.021632
         1998-01-12           1            0  0.001548 -0.031423
         1998-01-13           1            1  0.018010  0.028745
         1998-01-14           1            1  0.010230  0.006582

In [9]:
newsdf = pd.read_parquet(os.path.join(data_folder, 'news_headlines_v3.parquet'))

In [11]:
newsdf = newsdf.merge(df_price[['RET_5D_pos', 'RET_10D_pos']], left_on = ['SYMBOL', 'DATE'], right_on = ['SYMBOL', 'DATE'], how = 'left')

In [13]:
print(newsdf.shape)
print(newsdf.dropna(subset = ['RET_5D_pos', 'RET_10D_pos']).shape)

(1002870, 9)
(1002870, 9)


In [14]:
newsdf.to_parquet(os.path.join(data_folder, 'news_headlines_v3.2.parquet'))

: 